In [51]:
import sys 
import os
sys.path.append(os.path.abspath("/Users/madisonthantu/Desktop/COMS 6998/Final Project/recursive_LLMs"))
# import src.Measurement.globals
# from src.Measurement.measurement import Measurement
# sys.path.append(os.path.abspath("/Users/madisonthantu/Desktop/COMS 6998/Final Project/recursive_LLMs"))
# from src.utils import *
# globals.init()
# print(globals.API_URL)

import os
import sys
import argparse
from tqdm import tqdm
import pickle
import pandas as pd
import numpy as np


import torch, os, re, pandas as pd, json
# from sklearn.model_selection import train_test_split
from transformers import DataCollatorForLanguageModeling, GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments, AutoConfig
from datasets import Dataset, load_dataset

from huggingface_hub import login

In [57]:
dataset_names = ['news', 'reddit', 'dialogue']
dataset_paths = {
    'news': '/Users/madisonthantu/Desktop/COMS_6998/Final_Project/recursive_LLMs/Data/initial_datasets/news/sharded',
    'reddit': '/Users/madisonthantu/Desktop/COMS_6998/Final_Project/recursive_LLMs/Data/initial_datasets/reddit/sharded',
    'dialogue': '/Users/madisonthantu/Desktop/COMS_6998/Final_Project/recursive_LLMs/Data/initial_datasets/dialogue'
}

In [58]:
base_tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
separator = " TL;DR "
separator_length = len(base_tokenizer.tokenize(separator))
global_max_length = base_tokenizer.model_max_length
# src_length = global_max_length - separator_length - target_length
summary_column = 'summary'
padding = 'do_not_pad'
eos = '<|EOS|>'

In [59]:
def preprocess(sample):
    summary = sample[summary_column] + ' ' + eos
    labels = base_tokenizer(summary, padding=padding, truncation=False, return_attention_mask=False)#, return_tensors='pt')
    
    return labels

In [61]:
max_target_lengths = {}
percentiles = [50, 75, 90, 95, 99.5]

for dataset_name in dataset_names:
    print(dataset_name)
    dataset = load_dataset(dataset_paths[dataset_name], data_files='full_data.csv')['train']
    tokenized_dataset = dataset.map(
        preprocess,
        remove_columns=dataset.column_names,
    )
    tokenized_summ_lens = np.array([len(x) for x in tokenized_dataset['input_ids']])
    dataset_percentiles = np.percentile(tokenized_summ_lens, percentiles)
    max_target_lengths[dataset_name] = dataset_percentiles

news


Map:   0%|          | 0/38997 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1408 > 1024). Running this sequence through the model will result in indexing errors


reddit


Map:   0%|          | 0/21070 [00:00<?, ? examples/s]

dialogue


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/16369 [00:00<?, ? examples/s]

In [63]:
for i, p in enumerate(percentiles):
    print(f"Percentile {p} \n\tNews={max_target_lengths['news'][i]}\tReddit={max_target_lengths['reddit'][i]}\tDialogue={max_target_lengths['dialogue'][i]}")

Percentile 50 
	News=68.0	Reddit=28.0	Dialogue=28.0
Percentile 75 
	News=83.0	Reddit=37.0	Dialogue=39.0
Percentile 90 
	News=104.0	Reddit=49.0	Dialogue=51.0
Percentile 95 
	News=121.0	Reddit=59.0	Dialogue=59.0
Percentile 99.5 
	News=174.0	Reddit=96.0	Dialogue=75.0
